## 1. Setup & Data Preparation

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

In [3]:
data_dir = "UTKFace"
model_path = "models"

In [4]:
# Define the dataset class
class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transform = transforms
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.root_dir, image_name)
        image = Image.open(image_path).convert('RGB')
        
        # Extract age and gender from the filename
        age = int(image_name.split('_')[0])
        gender = int(image_name.split('_')[1])
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([age], dtype=torch.float32), torch.tensor([gender], dtype=torch.long)

# Define the transformation

train_transforms  = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(
    UTKFaceDataset(data_dir, transforms=test_transforms),
    [int(0.8 * len(os.listdir(data_dir))), int(0.2 * len(os.listdir(data_dir)))],
    generator=torch.Generator().manual_seed(42)
)

test_dataset.transform = test_transforms
train_dataset.transform = train_transforms
# over sample the training dataset to balance the classes
class_counts = pd.Series([int(f.split('_')[1]) for f in os.listdir(data_dir) if f.endswith('.jpg')]).value_counts()
max_class_count = class_counts.max()
repeat_count = max(1, max_class_count // len(train_dataset))
train_dataset = torch.utils.data.ConcatDataset([
    UTKFaceDataset(data_dir, transforms=train_transforms) for _ in range(repeat_count)
])


# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



## Deep CNNs

### Gender classification

In [5]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                                   padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
      
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

# Build Separable Convolutional Neural Network for gender classification
class GenderClassifierCnn(nn.Module):
    def __init__(self):
        super(GenderClassifierCnn, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x
        



In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
# Initialize the model, loss function, and optimizer
gender_classifier = GenderClassifierCnn()
gender_classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gender_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [8]:
from tqdm import tqdm

# Evaluate function: Calculate accuracy on the test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images in tqdm(test_loader):
            inputs, _, labels = images
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
    
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy


# Training function
def train_model(model, train_loader, criterion, optimizer, lr_scheduler=None ,num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        accuracy = 0.0
        for images in tqdm(train_loader):
            inputs, _, labels = images
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze())
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            accuracy += (outputs.argmax(dim=1) == labels.squeeze()).sum().item()
        if lr_scheduler:
            lr_scheduler.step()
        accuracy /= len(train_loader.dataset)
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")  
        evaluate_model(model, test_loader)


        

In [ ]:
# Train the model
train_model(gender_classifier, train_loader, criterion, optimizer, lr_scheduler ,num_epochs=30)

100%|██████████| 652/652 [03:07<00:00,  3.47it/s]


Epoch [1/30], Loss: 0.6298, Accuracy: 0.6516


100%|██████████| 131/131 [00:29<00:00,  4.47it/s]


Test Accuracy: 0.7159


100%|██████████| 652/652 [02:54<00:00,  3.74it/s]


...

Test Accuracy: 0.9254


100%|██████████| 652/652 [02:57<00:00,  3.68it/s]


Epoch [30/30], Loss: 0.2071, Accuracy: 0.9212


100%|██████████| 131/131 [00:30<00:00,  4.25it/s]

Test Accuracy: 0.9269


In [ ]:
# Save the model

if not os.path.exists(model_path):
    os.makedirs(model_path)
torch.save(gender_classifier.state_dict(), os.path.join(model_path, "gender_classifier.pth"))

## Convolutional Block Attention Module (CBAM)

### 1. Channel Attention Module

In [9]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        
        return self.sigmoid(out) * x 
    
    

### 2. Spatial Attention Module

In [10]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out = torch.max(x, dim=1, keepdim=True).values
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat)) * x

### 3. CBAM (Channel + Spatial)

In [11]:
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=8, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size=kernel_size)
    
    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        
        return x

### Channel Attention Gender Classifier

In [12]:
class GenderClassifierWithCA(nn.Module):
    def __init__(self):
        super(GenderClassifierWithCA, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.ca1 = ChannelAttention(32)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.ca2 = ChannelAttention(64)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.ca3 = ChannelAttention(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.ca1(F.relu(self.bn1(self.conv1(x)))))
        x = self.pool(self.ca2(F.relu(self.bn2(self.conv2(x)))))
        x = self.pool(self.ca3(F.relu(self.bn3(self.conv3(x)))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x
        

In [43]:
# Initialize the model, loss function, and optimizer with Channel Attention
CA_classifier = GenderClassifierWithCA()
CA_classifier.to(device)
criterion_CA = nn.CrossEntropyLoss()
optimizer_CA = optim.Adam(CA_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_CA = StepLR(optimizer_CA, step_size=10, gamma=0.1)


In [44]:
# Train the model with Channel Attention
train_model(CA_classifier, train_loader, criterion_CA, optimizer_CA, lr_scheduler=lr_scheduler_CA, num_epochs=30)

100%|██████████| 652/652 [03:07<00:00,  3.48it/s]


Epoch [1/30], Loss: 0.6169, Accuracy: 0.6651


100%|██████████| 131/131 [00:30<00:00,  4.29it/s]


Test Accuracy: 0.7317


100%|██████████| 652/652 [03:00<00:00,  3.61it/s]


...

Test Accuracy: 0.9509


100%|██████████| 652/652 [03:48<00:00,  2.86it/s]


Epoch [30/30], Loss: 0.1851, Accuracy: 0.9301


100%|██████████| 131/131 [00:41<00:00,  3.19it/s]

Test Accuracy: 0.9497


In [47]:
# Save the model with Channel Attention
torch.save(CA_classifier.state_dict(), os.path.join(model_path, "CA_gender_classifier.pth"))

### Spatial Attention Gender Classifier

In [13]:
# Build a model with Spatial Attention
class GenderClassifierWithSA(nn.Module):
    def __init__(self):
            super(GenderClassifierWithSA, self).__init__()
            # Depthwise separable convolutional layers
            self.conv1 = DepthwiseSeparableConv(3, 32)
            self.bn1 = nn.BatchNorm2d(32)
            self.sa1 = SpatialAttention()
            
            self.conv2 = DepthwiseSeparableConv(32, 64)
            self.bn2 = nn.BatchNorm2d(64)
            self.ca2 = ChannelAttention(64)
            self.sa2 = SpatialAttention()
            
            self.conv3 = DepthwiseSeparableConv(64, 128)
            self.bn3 = nn.BatchNorm2d(128)
            self.ca3 = ChannelAttention(128)
            self.sa3 = SpatialAttention()
            
            # Fully connected layers
            self.fc1 = nn.Linear(128 * 16 * 16, 256)
            self.bn_fc1 = nn.BatchNorm1d(256)
            self.fc2 = nn.Linear(256, 2)
            self.dropout = nn.Dropout(0.5)
            self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.sa1(F.relu(self.bn1(self.conv1(x)))))
        x = self.pool(self.sa2(F.relu(self.bn2(self.conv2(x)))))
        x = self.pool(self.sa3(F.relu(self.bn3(self.conv3(x)))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x
        
        

In [58]:
# Initialize the model, loss function, and optimizer with Spatial Attention
SA_classifier = GenderClassifierWithSA()
SA_classifier.to(device)
criterion_SA = nn.CrossEntropyLoss()
optimizer_SA = optim.Adam(SA_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_SA = StepLR(optimizer_SA, step_size=10, gamma=0.1)


In [59]:
# Train the model with Spatial Attention
train_model(SA_classifier, train_loader, criterion_SA, optimizer_SA, lr_scheduler=lr_scheduler_SA, num_epochs=30)

100%|██████████| 652/652 [03:48<00:00,  2.85it/s]


Epoch [1/30], Loss: 0.6174, Accuracy: 0.6631


100%|██████████| 131/131 [00:36<00:00,  3.56it/s]


Test Accuracy: 0.7363


100%|██████████| 652/652 [03:15<00:00,  3.34it/s]


...

Test Accuracy: 0.9739


100%|██████████| 652/652 [02:56<00:00,  3.68it/s]


Epoch [30/30], Loss: 0.1241, Accuracy: 0.9543


100%|██████████| 131/131 [00:29<00:00,  4.39it/s]

Test Accuracy: 0.9748


In [60]:
# Save the model with
torch.save(SA_classifier.state_dict(), os.path.join(model_path, "SA_gender_classifier.pth"))

### CBAM Gender Classifier

In [ ]:
# Build a model with CBAM
class GenderClassifierWithCBAM(nn.Module):
    def __init__(self):
        super(GenderClassifierWithCBAM, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.cbam1 = CBAM(32)
        self.dropout1 = nn.Dropout2d(0.3)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.cbam2 = CBAM(64)
        self.dropout2 = nn.Dropout2d(0.3)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.cbam3 = CBAM(128)
        self.dropout3 = nn.Dropout2d(0.3)
        
        self.conv4 = DepthwiseSeparableConv(128, 64)
        self.bn4 = nn.BatchNorm2d(64)
        self.cbam4 = CBAM(64)
        self.dropout4 = nn.Dropout2d(0.3)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.cbam1(F.relu(self.bn1(self.conv1(x)))))
        x = self.pool(self.cbam2(F.relu(self.bn2(self.conv2(x)))))
        x = self.pool(self.cbam3(F.relu(self.bn3(self.conv3(x)))))
        x = self.pool(self.cbam4(F.relu(self.bn4(self.conv4(x)))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x

In [15]:
# Initialize the model, loss function, and optimizer with CBAM
CBAM_classifier = GenderClassifierWithCBAM()
CBAM_classifier.to(device)
criterion_CBAM = nn.CrossEntropyLoss()
optimizer_CBAM = optim.Adam(CBAM_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_CBAM = StepLR(optimizer_CBAM, step_size=10, gamma=0.1)



In [17]:
# Train the model with CBAM
train_model(CBAM_classifier, train_loader, criterion_CBAM, optimizer_CBAM, lr_scheduler=lr_scheduler_CBAM, num_epochs=35)

100%|██████████| 652/652 [03:49<00:00,  2.84it/s]


Epoch [1/35], Loss: 0.6004, Accuracy: 0.6771


100%|██████████| 131/131 [00:30<00:00,  4.25it/s]


Test Accuracy: 0.7473


100%|██████████| 652/652 [03:46<00:00,  2.88it/s]


...

Test Accuracy: 0.9803


100%|██████████| 652/652 [02:59<00:00,  3.63it/s]


Epoch [35/35], Loss: 0.0950, Accuracy: 0.9657


100%|██████████| 131/131 [00:30<00:00,  4.28it/s]

Test Accuracy: 0.9806


In [18]:
# Save the model with CBAM
torch.save(CBAM_classifier.state_dict(), os.path.join(model_path, "CBAM_gender_classifier.pth"))

## Squeeze-and-Excitation (SE) Block

In [25]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) # Calculate the average weight of each channel
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size() # Batch size and number of channels
        y = self.global_avg_pool(x).view(b, c) # [B, C]
        y = self.fc(y).view(b, c, 1, 1) # [B, C, 1, 1]
        
        return x * y.expand_as(x)  # Scaled channel wise
        


In [26]:
class GenderClassifierWithSE(nn.Module):
    def __init__(self):
        super(GenderClassifierWithSE, self).__init__()
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.se1 = SEBlock(32)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.se2 = SEBlock(64)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.se3 = SEBlock(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.se1(F.relu(self.bn1(self.conv1(x)))))
        x = self.pool(self.se2(F.relu(self.bn2(self.conv2(x)))))
        x = self.pool(self.se3(F.relu(self.bn3(self.conv3(x)))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x
        


In [27]:
# Initialize the models with SE 
SE_classifier = GenderClassifierWithSE()
SE_classifier.to(device)
criterion_SE = nn.CrossEntropyLoss()
optimizer_SE = optim.Adam(SE_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_SE = StepLR(optimizer_SE, step_size=10, gamma=0.1)


In [28]:
# Train the model with SE
train_model(SE_classifier, train_loader, criterion_SE, optimizer_SE, lr_scheduler=lr_scheduler_SE, num_epochs=35)

100%|██████████| 652/652 [03:47<00:00,  2.86it/s]


Epoch [1/35], Loss: 0.6229, Accuracy: 0.6580


100%|██████████| 131/131 [00:39<00:00,  3.28it/s]


Test Accuracy: 0.7181


100%|██████████| 652/652 [03:54<00:00,  2.79it/s]


...

Test Accuracy: 0.9312


100%|██████████| 652/652 [02:57<00:00,  3.68it/s]


Epoch [35/35], Loss: 0.2258, Accuracy: 0.9141


100%|██████████| 131/131 [00:30<00:00,  4.27it/s]

Test Accuracy: 0.9312


In [29]:
# Save the model with SE
torch.save(SE_classifier.state_dict(), os.path.join(model_path, "SE_Gender_classifier.pth"))

### SE Block with Spatial Attention

In [30]:
class GenderClassifierWithSAandSE(nn.Module):
    def __init__(self):
        super(GenderClassifierWithSAandSE, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.se1 = SEBlock(32)
        self.sa1 = SpatialAttention()

        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.se2 = SEBlock(64)
        self.sa2 = SpatialAttention()
        
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.se3 = SEBlock(128)
        self.sa3 = SpatialAttention()
        
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(self.sa1(self.se1(F.relu(self.bn1(self.conv1(x))))))
        x = self.pool(self.sa2(self.se2(F.relu(self.bn2(self.conv2(x))))))
        x = self.pool(self.sa3(self.se3(F.relu(self.bn3(self.conv3(x))))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x

In [31]:
# Initialize the model with SA and SE
SA_SE_classifier = GenderClassifierWithSAandSE()
SA_SE_classifier.to(device)
criterion_SA_SE = nn.CrossEntropyLoss()
optimizer_SA_SE = optim.Adam(SA_SE_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_SA_SE = StepLR(optimizer_SA_SE, step_size=10, gamma=0.1)

In [32]:
# Train the model with SA and SE 
train_model(SA_SE_classifier, train_loader, criterion_SA_SE, optimizer_SA_SE, lr_scheduler=lr_scheduler_SA_SE, num_epochs=35)

100%|██████████| 652/652 [02:56<00:00,  3.69it/s]


Epoch [1/35], Loss: 0.6151, Accuracy: 0.6616


100%|██████████| 131/131 [00:29<00:00,  4.45it/s]


Test Accuracy: 0.7356


100%|██████████| 652/652 [03:01<00:00,  3.60it/s]


...

Test Accuracy: 0.9847


100%|██████████| 652/652 [02:57<00:00,  3.67it/s]


Epoch [35/35], Loss: 0.0879, Accuracy: 0.9713


100%|██████████| 131/131 [00:29<00:00,  4.40it/s]

Test Accuracy: 0.9842


In [33]:
# Save the model with SE and SA
torch.save(SA_SE_classifier.state_dict(), os.path.join(model_path, "SA_and_SE_Gender_classifier.pth"))

### SE Block with CBAM

In [34]:
class GenderClassifierWithCBAMandSE(nn.Module):
    def __init__(self):
        super(GenderClassifierWithCBAMandSE, self).__init__()
        # Depthwise separable convolutional layers
        self.conv1 = DepthwiseSeparableConv(3, 32)
        self.bn1 = nn.BatchNorm2d(32)
        self.cbam1 = CBAM(32)
        self.se1 = SEBlock(32)
        
        self.conv2 = DepthwiseSeparableConv(32, 64)
        self.bn2 = nn.BatchNorm2d(64)
        self.cbam2 = CBAM(64)
        self.se2 = SEBlock(64)
        
        self.conv3 = DepthwiseSeparableConv(64, 128)
        self.bn3 = nn.BatchNorm2d(128)
        self.cbam3 = CBAM(128)
        self.se3 = SEBlock(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 2)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pool(self.cbam1(self.se1(F.relu(self.bn1(self.conv1(x))))))
        x = self.pool(self.cbam2(self.se2(F.relu(self.bn2(self.conv2(x))))))
        x = self.pool(self.cbam3(self.se3(F.relu(self.bn3(self.conv3(x))))))
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        # Fully connected layers with ReLU activation and dropout
        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        # Output layer
        x = self.fc2(x)
        return x




In [35]:
# Initialize the model with CBAM and SE
CBAM_SE_classifier = GenderClassifierWithCBAMandSE()
CBAM_SE_classifier.to(device)
criterion_CBAM_SE = nn.CrossEntropyLoss()
optimizer_CBAM_SE = optim.Adam(CBAM_SE_classifier.parameters(), lr=0.0001, weight_decay=1e-5)
lr_scheduler_CBAM_SE = StepLR(optimizer_CBAM_SE, step_size=10, gamma=0.1)

In [36]:
# Train the model with CBAM and SE
train_model(CBAM_SE_classifier, train_loader, criterion_CBAM_SE, optimizer_CBAM_SE, lr_scheduler=lr_scheduler_CBAM_SE, num_epochs=35)

100%|██████████| 652/652 [02:55<00:00,  3.71it/s]


Epoch [1/35], Loss: 0.6236, Accuracy: 0.6515


100%|██████████| 131/131 [00:30<00:00,  4.34it/s]


Test Accuracy: 0.7276


100%|██████████| 652/652 [02:55<00:00,  3.72it/s]


...

Test Accuracy: 0.9777


100%|██████████| 652/652 [02:55<00:00,  3.71it/s]


Epoch [35/35], Loss: 0.1032, Accuracy: 0.9653


100%|██████████| 131/131 [00:30<00:00,  4.33it/s]

Test Accuracy: 0.9779


In [37]:
# Save the model with CBAM and SE
torch.save(CBAM_SE_classifier.state_dict(), os.path.join(model_path, "CBAM_and_SE_Gender_classifier.pth"))